To build/rebuild run: 
```
jbuilder build src-core/core.cma
jbuilder build src-core-pp/core_pp.cma
```

In [1]:
#directory "_build/default/src-core/";;
#directory "_build/default/src-core-pp/";;
#load "_build/default/src-core/core.cma";;
#load "_build/default/src-core-pp/core_pp.cma";;

In [2]:
let message_of_string str = str
    |> Stream.of_string 
    |> Parser_utils.split_into_key_value '|'
    |> Parser_utils.split_into_messages
    |> Stream.peek
    |> function Some m -> m | _ -> failwith "WAT"

val message_of_string : string -> (string * string) list = <fun>


In [3]:
let message = message_of_string "42=42|43=43|45=45|1=3|2=9|3=9|2=8|3=8|2=7|3=7|12=12|13=13|"

val message : (string * string) list =
  [("42", "42"); ("43", "43"); ("45", "45"); ("1", "3"); ("2", "9");
   ("3", "9"); ("2", "8"); ("3", "8"); ("2", "7"); ("3", "7"); ("12", "12");
   ("13", "13")]


### Checking my custom list utils for the parser

`Parser_utils.take` extracts a value given a tag, leavig rest of the list unmodified.

In [4]:
Parser_utils.take "1" message;;

- : string option * (string * string) list =
(Some "3",
 [("42", "42"); ("43", "43"); ("45", "45"); ("2", "9"); ("3", "9");
  ("2", "8"); ("3", "8"); ("2", "7"); ("3", "7"); ("12", "12"); ("13", "13")])


`Parser_utils.split_on_tag` splits message at a given tag

In [5]:
let leading, groups = Parser_utils.split_on_tag "1" message;;

val leading : (string * string) list =
  [("42", "42"); ("43", "43"); ("45", "45")]
val groups : (string * string) list =
  [("1", "3"); ("2", "9"); ("3", "9"); ("2", "8"); ("3", "8"); ("2", "7");
   ("3", "7"); ("12", "12"); ("13", "13")]


In [6]:
List.tl groups |> Parser_utils.cut_on_separator

- : (string * string) list list =
[[("2", "9"); ("3", "9")]; [("2", "8"); ("3", "8")];
 [("2", "7"); ("3", "7"); ("12", "12"); ("13", "13")]]


### Making a mock parser

In [7]:
type test_block = {
    tb_2 : int;
    tb_3 : int option
};; 

type test_block = { tb_2 : int; tb_3 : int option; }


In [8]:
let parse_block msg =
    let open Parse_base_types in
    let open Parser_utils.Parser in
    req msg "2" parse_int @@ fun msg tb_2 ->
    opt msg "3" parse_int @@ fun msg tb_3 ->
    ParseSuccess { tb_2; tb_3 } , msg

val parse_block :
  (string * string) list ->
  test_block Parser_utils.Parser.t * (string * string) list = <fun>


In [9]:
parse_block [("2", "3"); ("3", "2"); ("4", "3")];;

- : test_block Parser_utils.Parser.t * (string * string) list =
(Parser_utils.Parser.ParseSuccess {tb_2 = 3; tb_3 = Some 2}, [("4", "3")])


In [13]:
type test_message = {
    tm_43 : int;
    tm_repeating : test_block list
}

type test_message = { tm_43 : int; tm_repeating : test_block list; }


In [15]:
let parse_test msg = 
    let open Parse_base_types in
    let open Parser_utils.Parser in 
    req msg "43" parse_int @@ fun msg tm_43 -> 
    repeating msg "1" parse_block @@ fun msg tm_repeating ->
    ParseSuccess { tm_43; tm_repeating }, msg

val parse_test :
  (string * string) list ->
  test_message Parser_utils.Parser.t * Parser_utils.Parser.msg = <fun>


### Testing the parser and error reporing

In [16]:
"42=42|43=43|45=45|1=3|2=9|3=9|2=8|3=8|2=7|3=7|12=12|13=13|" 
|> message_of_string |> parse_test 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(Parser_utils.Parser.ParseSuccess
  {tm_43 = 43;
   tm_repeating =
    [{tb_2 = 9; tb_3 = Some 9}; {tb_2 = 8; tb_3 = Some 8};
     {tb_2 = 7; tb_3 = Some 7}]},
 [("42", "42"); ("45", "45"); ("12", "12"); ("13", "13")])


In [17]:
"42=42|43=43|45=45|1=3|2=9|3=9|2=8|2=7|3=7|12=12|13=13|" 
|> message_of_string |> parse_test 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(Parser_utils.Parser.ParseSuccess
  {tm_43 = 43;
   tm_repeating =
    [{tb_2 = 9; tb_3 = Some 9}; {tb_2 = 8; tb_3 = None};
     {tb_2 = 7; tb_3 = Some 7}]},
 [("42", "42"); ("45", "45"); ("12", "12"); ("13", "13")])


In [18]:
"42=42|43=43|45=45|1=3|2=9|3=9|2=8|5=2|3=8|2=7|3=7|12=12|13=13|" 
|> message_of_string |> parse_test 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(Parser_utils.Parser.RepeatingGroupOutOfOrder "1", [])


In [19]:
"42=42|43=43|45=45|1=3|2=8|3=8|2=7|3=7|12=12|13=13|" 
|> message_of_string |> parse_test 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(Parser_utils.Parser.IncorrectNumInGroupCount "1", [])
